# Water Pump Classfication

**Research Question**
* Identify which factors affect waterpumps and in what way,and predict waterpump's status so that it can help authorities improve maintenance operations and ensure availability of water to all communities in Tanzania

**DataSet**

* The dataset is from the Taarifa, which aggregates data from the Tanzania Ministry of Water.
[Pump it Up: Data Mining the Water Table](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/24/)

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Load Data & Inspect
The dataset has following:
* Training set values
* Trainng set labels
* Test set values

I will be working on Training data

In [ ]:
# Load training data
df_pump = pd.read_csv('../input/pump-it-up-data-mining-the-water-table/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_values.csv')
df_pump

In [ ]:
# Load training labels
df_label = pd.read_csv('../input/pump-it-up-data-mining-the-water-table/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv')
df_label

In [ ]:
#plot the target variable to view
plt.subplots(figsize=(10,6))
ax = sns.countplot(x=df_label['status_group'])
for p in ax.patches:
        ax.annotate('{:.2f}%'.format(p.get_height()*100/len(df_label)), (p.get_x()+0.3, p.get_height()*0.5))

> There are 3 types of statuses with different percentages. 
Functional needs repair is lowest. 
The visualization indicates that classess are imbalanced

In [ ]:
# Merge labels with main data for ease in subsequent data clearning & transformation processes
df_pump = df_pump.merge(df_label, how='inner', on='id')

In [ ]:
#Get info on columns, and data types
df_pump.info()

In [ ]:
# Check unique count of respective columns. 
# This will help in identifying categorical & non-categorical data
df_pump.nunique()

In [ ]:
df_pump.isnull().sum()/len(df_pump)*100


In [ ]:
# Get visual sense of null data
plt.subplots(figsize=(15,8))
sns.heatmap(df_pump.isnull())

# Cleaning & Transforming Data

In [ ]:
# Describe all columns to get top frequencies, min/max, means, deviations
df_pump.describe(include='all').T

**Observations:**
* Certain columns are either grouped or encoded values of another column e.g: quantity vs quantity_group, region vs region_code
* Some look like free texts or too many nulls, or just having zeros. e.g: wpt_name, scheme_name, amount_tsh, num_private
* Some categorical variables have too many unique values

**Strategy**
1. Keep one of the columns which are either grouped or encoded
1. Drop columns with mostly nulls or zeros
1. Keep top 10-15 of the non-categorical data like funder, installer etc.


In [ ]:
#df_pump[df_pump.amount_tsh==0]['amount_tsh'].count()
#len(df_pump[df_pump.public_meeting.isnull()==True]) -> 3334
#df_pump.recorded_by.nunique()

#df_pump.scheme_name
#df_pump.scheme_name.nunique()
#len(df_pump[df_pump.scheme_name.isnull()==True]) -> 28166

#df_pump.permit -> True/False
#len(df_pump[df_pump.permit.isnull()==True]) -> 3056

#df_pump.construction_year.nunique() -> 55 years
#max(df_pump.construction_year) -> 2013
#len(df_pump[df_pump.construction_year==0]) -> 20709

#df_pump.construction_year.nunique() -> 55 years
#max(df_pump.construction_year) -> 2013
#len(df_pump[df_pump.construction_year==0]) -> 20709


In [ ]:
# Removing columns as:
    # amount_tsh -> 75% are zeros and has outliers
    # wpt_name -> free text, mostly nuls
    # latitude, longitude -> as other geographic locations available
    # num_private -> no clue, almost no data
    # subvillage -> will use district code
    # region -> will use region code
    # lga, ward -> yet another geographic locations
    # recorded_by -> just one value
    # scheme_name -> more than 50% null, free text, will use scheme management
    # extraction_type & extraction_type_class -> will use extraction_type_group
    # management_group -> will use management
    # payment -> will use payment_type
    # water_quality -> will use quality_group
    # quantity -> will use quantity_group
    # source_type, source_class -> will use source
    # waterpoint_type -> will use waterpoint_type_group
    
df_pump = df_pump.drop(['amount_tsh','wpt_name','latitude','longitude','num_private','subvillage',
                        'region','lga','ward','recorded_by','scheme_name','extraction_type',
                        'extraction_type_class','management_group','payment','water_quality',
                        'quantity','source_type','source_class','waterpoint_type'], axis=1)    

**Drop NaN values**

In [ ]:
df_pump = df_pump.dropna()
df_pump

**Limiting funder & installer categorical values**

In [ ]:
# Current counts of funder & installer
[df_pump['funder'].nunique(), df_pump['installer'].nunique()]

In [ ]:
df_fgb = df_pump.groupby(['funder'], as_index=False).count()[['funder', 'id']]
df_fgb

In [ ]:
# Funders with less than 5 pumps, will be renamed to Other
other_funders = df_fgb[df_fgb.id<5].funder.unique()
df_pump['funder'] = df_pump['funder'].apply(lambda x : x if (x not in other_funders)  else 'Other')

In [ ]:
# Installers with less than 5 pumps, will be renamed to Other
df_igb = df_pump.groupby(['installer'], as_index=False).count()[['installer', 'id']]
other_installers = df_igb[df_igb.id<5].installer.unique()
df_pump['installer'] = df_pump['installer'].apply(lambda x : x if (x not in other_installers)  else 'Other')

In [ ]:
# Cleanup
del df_fgb
del df_igb

# View reduced counts
[df_pump['funder'].nunique(), df_pump['installer'].nunique()]

# Analyze Relations & Create New Features
# 
Making some assumptions about the features that could play key role:
* Age of the waterpoint -> older ones might need more repairs
* Population around pump -> probably dense population around waterpoint will cause heavy usage and require more maintenance
* How the waterpoint is managed
* The source of the water
* Payment type -> what or how the water costs are paid
* Type of Waterpoint -> Probably certain types need more maintenance

**Analyzing Relation with Age of the Waterpoint**

In [ ]:
df_pump['date_recorded'].max()

In [ ]:
df_pump['construction_year'].max()

In [ ]:
# We have date fields as date_recorded and construction_year, will consider the later one
df_pump['construction_year'].unique()

In [ ]:
# create a new feature 'age' relative to the max construction year
df_pump['age'] = df_pump['construction_year'].apply(lambda x: (2013 - x) if x != 0 else 0 )

# review relation with recorded date
df_pump[['age', 'construction_year', 'date_recorded']].drop_duplicates()

In [ ]:
df_pump.age.describe()

In [ ]:
# There are many age values, will group them for better analysis
def get_age_group(x):
    if x <= 17:
        return 'new'
    elif x > 17 and x <=34:
        return 'middle'
    else:
        return 'old'

# Create new feature age_group
df_pump['age_group'] = df_pump['age'].apply(get_age_group)

# Visualize Relation
plt.subplots(figsize=(12,8))
sns.countplot(x=df_pump['age_group'], hue=df_pump['status_group'])
plt.title('Age Group Vs Water-pump Status')
plt.show()

**Analyzing Relation with Population of the Waterpoint**

In [ ]:
df_pump.population.describe()

In [ ]:
# Max value and value of 75% population indicates outliers as well as very low population mostly
# Visualize through boxplot
sns.boxplot(x=df_pump.population)

In [ ]:
print('Outlier for population =', df_pump[df_pump.population > 10000].population.tolist() )

In [ ]:
# Create Population Group to better assess the sparse data
def get_population_group(x):
    if x <= 500:
        return 'low'
    else:
        return 'dense'
    
df_pump['population_group'] = df_pump['population'].apply(get_population_group)

# Visualize Relation
plt.subplots(figsize=(12,8))
sns.countplot(x=df_pump['population_group'], hue=df_pump['status_group'])
plt.title('Population Group Vs Water-pump Status')
plt.show()

> > It seems there isn't any obvious relation of the status of the waterpoint and population around it

**Exploring relation with other variables: Managment, Payment Type, Source & Waterpoint_Type**

In [ ]:
def plot_counts(featureX):
    plt.subplots(figsize=(18,8))
    sns.set(font_scale=1)
    sns.countplot(x=df_pump[featureX], hue=df_pump['status_group'])
    plt.title(f'{featureX} Vs Water-pump Status')
    plt.show()
    return

# Management of Waterpoint
plot_counts('management')

# Payment Type of Waterpoint
plot_counts('payment_type')

# Water source of Waterpoint
plot_counts('source')

# Water source of Waterpoint
plot_counts('waterpoint_type_group')

In [ ]:
from sklearn.feature_selection import SelectKBest

# Encoding Categorical Variables 

In [ ]:
from sklearn import preprocessing

encoder = preprocessing.LabelEncoder()
categorical_features = ['funder','installer','basin', 'public_meeting', 'scheme_management', 'permit', 
                        'extraction_type_group', 'management', 'payment_type', 'quality_group', 'quantity_group', 
                        'source', 'waterpoint_type_group', 'age_group', 'population_group', 'status_group']

# Copy status group labels
df_pump['status_group_labels'] = df_pump['status_group']

# Encode values
df_pump[categorical_features] = df_pump[categorical_features].apply(encoder.fit_transform)

In [ ]:
df_pump.describe(include='all').T

In [ ]:
# Visualize Correlation with features Matrix 
corrMatrix = df_pump[['funder', 'gps_height', 'installer', 'basin', 'region_code', 'district_code', 
            'scheme_management', 'permit', 'extraction_type_group', 'management', 'payment_type', 'quality_group', 'quantity_group', 'source',
            'waterpoint_type_group', 'construction_year', 'age_group', 'population_group', 'status_group']].corr()
plt.subplots(figsize=(18,12))
sns.heatmap(corrMatrix, annot=True, fmt='.1g')
plt.title('Correlation of Features Matrix')
plt.show()

# Train Model & Predict

In [ ]:
#Prepare list of features for training and target for prediction
features = ['funder', 'gps_height', 'installer', 'basin', 'region_code', 'district_code', 
            'scheme_management', 'permit', 'extraction_type_group', 'management', 'payment_type', 'quality_group', 'quantity_group', 'source',
            'waterpoint_type_group', 'age_group', 'population_group'] 

target = ['status_group']
df_pump_X = df_pump[features]
df_pump_Y = df_pump[target]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report



In [ ]:
# random_state = 1, to get same split in case of re-run
# startify = target, to get split containing each type of class proportional to origingal dataset
X_train, X_test, y_train, y_test = train_test_split(df_pump_X, df_pump_Y, 
                                                    random_state=1, stratify= df_pump_Y, 
                                                    test_size = 0.33)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

In [ ]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train.values.ravel())

In [ ]:
y_predicted = rf_classifier.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_predicted)

In [ ]:
print(classification_report(y_test, y_predicted))

# Tuning Hyperparameter of Classifier

**Applying Grid Search**

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators' : [301,401,501],
    'max_depth' : [11,21,31],    
    'max_features' : ['sqrt','log2']
}

cv = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, n_jobs=-1)
cv.fit(X_train, y_train.values.ravel())

In [ ]:
cv.best_params_

In [ ]:
# Predict again with the values of best parameters
rf_classifier_bst = RandomForestClassifier(n_estimators=501, n_jobs=-1,max_depth=21, max_features='log2', bootstrap=True, criterion='gini')
rf_classifier_bst.fit(X_train, y_train.values.ravel())
y_predict_bst = rf_classifier_bst.predict(X_test)

# Reviewing Model Performance

In [ ]:
# Confusion Matrix -> Review where model made mistakes 
# Like False positives, False Negatives etc.

cnf_matrix = confusion_matrix(y_test, y_predict_bst)
cnf_matrix

In [ ]:
cnf_matrix = cnf_matrix.astype('float')  / cnf_matrix.sum(axis=1)[:, np.newaxis]

# Build the plot
plt.figure(figsize=(12,7))
sns.set(font_scale=1.2)
sns.heatmap(cnf_matrix, annot=True, cmap=plt.cm.Greens, linewidths=0.2)

# Add labels to the plot
class_names = ['Functional', 'Needs Repair', 'Non-Functional']
tick_marks = np.arange(len(class_names))
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks, class_names, rotation=25)
plt.yticks(tick_marks2, class_names, rotation=0)
plt.xlabel('Predicted Status')
plt.ylabel('True Status')
plt.title('Confusion Matrix Normalized for predicted Waterpump statuses')
plt.show()

In [ ]:
# Get Summary Report of Prediction Metrices
print(classification_report(y_test, y_predict_bst, target_names=class_names))

In [ ]:
# Formatting Classification Report
clf_rpt = classification_report(y_test, y_predict_bst, target_names=class_names, output_dict=True)

# .iloc[:-1, :] to exclude support
plt.figure(figsize=(12,7))
sns.heatmap(pd.DataFrame(clf_rpt).iloc[:-1, :].T, annot=True)
plt.title ('Classification Report')
plt.show()